In [2]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()




from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()
print(len(train_cands),len(dev_cands))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
8272 888


In [52]:
# from load_external_annotations import load_external_labels

# load_external_labels(session, ChemicalDisease, split=2, annotator='gold')

from snorkel.annotations import load_gold_labels
# L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

#L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
# L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
# gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
gold_labels_dev = [L[0,0] for L in L_gold_dev]

gold_labels_test = [L[0,0] for L in L_gold_test]

from snorkel.learning.utils import MentionScorer

print(gold_labels_dev.count(1),gold_labels_dev.count(0))
print(gold_labels_test.count(1),gold_labels_test.count(0))
print(len(gold_labels_dev),len(gold_labels_test))

AnnotatorLabels created: 0
296 592
1505 3115
888 4620


In [3]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [4]:
import bz2
from six.moves.cPickle import load

with bz2.BZ2File('data/ctd.pkl.bz2', 'rb') as ctd_f:
    ctd_unspecified, ctd_therapy, ctd_marker = load(ctd_f)

In [69]:
##### Discrete #########

def cand_in_ctd_unspecified(c):
    return 1 if c.get_cids() in ctd_unspecified else 0

def cand_in_ctd_therapy(c):
    return 1 if c.get_cids() in ctd_therapy else 0

def cand_in_ctd_marker(c):
    return 1 if c.get_cids() in ctd_marker else 0


def LF_in_ctd_unspecified(c):
    if(cand_in_ctd_unspecified(c)==1):
        return (-1,1)
    else:
        return (0,0)

def LF_in_ctd_therapy(c):
    if(cand_in_ctd_therapy(c)==1):
        return (-1,1)
    else:
        return (0,0)

def LF_in_ctd_marker(c):
    if(cand_in_ctd_marker(c)==1):
        return (1,1)
    else:
        return (0,0)

import re
from snorkel.lf_helpers import (
    get_tagged_text,
    rule_regex_search_tagged_text,
    rule_regex_search_btw_AB,
    rule_regex_search_btw_BA,
    rule_regex_search_before_A,
    rule_regex_search_before_B,
)

# List to parenthetical
def ltp(x):
    return '(' + '|'.join(x) + ')'

def LF_induce(c):
    return (1,1) if re.search(r'{{A}}.{0,20}induc.{0,20}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

causal_past = ['induced', 'caused', 'due']
def LF_d_induced_by_c(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + '.{0,9}(by|to).{0,50}', 1)==1):
        return (1,1)
    return (0,0)

def LF_d_induced_by_c_tight(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + ' (by|to) ', 1)==1):
        return (1,1)
    return (0,0)

def LF_induce_name(c):
    return (1,1) if 'induc' in c.chemical.get_span().lower() else (0,0)     

causal = ['cause[sd]?', 'induce[sd]?', 'associated with']
def LF_c_cause_d(c):
    return (1,1) if (
        re.search(r'{{A}}.{0,50} ' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
        and not re.search('{{A}}.{0,50}(not|no).{0,20}' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
    ) else (0,0)

treat = ['treat', 'effective', 'prevent', 'resistant', 'slow', 'promise', 'therap']
def LF_d_treat_c(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_treat_d(c):
    if (rule_regex_search_btw_AB(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_treat_d(c):
    if (rule_regex_search_before_B(c, ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_treat_d_wide(c):
    if (rule_regex_search_btw_AB(c, '.{0,200}' + ltp(treat) + '.{0,200}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_d(c):
    return (1,1) if ('{{A}} {{B}}' in get_tagged_text(c)) else (0,0)

def LF_c_induced_d(c):
    return (1,1) if (
        ('{{A}} {{B}}' in get_tagged_text(c)) and 
        (('-induc' in c[0].get_span().lower()) or ('-assoc' in c[0].get_span().lower()))
        ) else (0,0)

def LF_improve_before_disease(c):
    if(rule_regex_search_before_B(c, 'improv.*', -1)==-1):
        return (-1,1)
    return (0,0)

pat_terms = ['in a patient with ', 'in patients with']
def LF_in_patient_with(c):
    return (-1,1) if re.search(ltp(pat_terms) + '{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

uncertain = ['combin', 'possible', 'unlikely']
def LF_uncertain(c):
    if (rule_regex_search_before_A(c, ltp(uncertain) + '.*', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_induced_other(c):
    if (rule_regex_search_tagged_text(c, '{{A}}.{20,1000}-induced {{B}}', -1)==-1):
        return (-1,1)
    return (0,0)  

def LF_far_c_d(c):
    if (rule_regex_search_btw_AB(c, '.{100,5000}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_far_d_c(c):
    if (rule_regex_search_btw_BA(c, '.{100,5000}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_risk_d(c):
    if (rule_regex_search_before_B(c, 'risk of ', 1)==1):
        return (1,1)
    return (0,0)

def LF_develop_d_following_c(c):
    return (1,1) if re.search(r'develop.{0,25}{{B}}.{0,25}following.{0,25}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)

procedure, following = ['inject', 'administrat'], ['following']
def LF_d_following_c(c):
    return (1,1) if re.search('{{B}}.{0,50}' + ltp(following) + '.{0,20}{{A}}.{0,50}' + ltp(procedure), get_tagged_text(c), flags=re.I) else (0,0)

def LF_measure(c):
    return (-1,1) if re.search('measur.{0,75}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)

def LF_level(c):
    return (-1,1) if re.search('{{A}}.{0,25} level', get_tagged_text(c), flags=re.I) else (0,0)

def LF_neg_d(c):
    return (-1,1) if re.search('(none|not|no) .{0,25}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

WEAK_PHRASES = ['none', 'although', 'was carried out', 'was conducted',
                'seems', 'suggests', 'risk', 'implicated',
               'the aim', 'to (investigate|assess|study)']

WEAK_RGX = r'|'.join(WEAK_PHRASES)

def LF_weak_assertions(c):
    return (-1,1) if re.search(WEAK_RGX, get_tagged_text(c), flags=re.I) else (0,0)


def LF_ctd_marker_c_d(c):
    l,s = LF_c_d(c)
    cl = cand_in_ctd_marker(c)
    return (l*cl,s*cl)

def LF_ctd_marker_induce(c):
    l1,s1 = LF_c_induced_d(c)
    l2,s2 = LF_d_induced_by_c_tight(c)
    cl = cand_in_ctd_marker(c)
    return ((l1 or l2)*cl,max(s1,s2)*cl)

def LF_ctd_therapy_treat(c):
    l,s = LF_c_treat_d_wide(c)
    cl = cand_in_ctd_therapy(c)
    return (l*cl,s*cl)

def LF_ctd_unspecified_treat(c):
    l,s = LF_c_treat_d_wide(c)
    cl = cand_in_ctd_unspecified(c)
    return (l*cl,s*cl)

def LF_ctd_unspecified_induce(c):
    l1,s1 = LF_c_induced_d(c)
    l2,s2 = LF_d_induced_by_c_tight(c)
    cl = cand_in_ctd_unspecified(c)
    return ((l1 or l2)*cl,max(s1,s2)*cl)


# def LF_ctd_marker_c_d(c):
#     return LF_c_d(c) * cand_in_ctd_marker(c)

# def LF_ctd_marker_induce(c):
#     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_marker(c)

# def LF_ctd_therapy_treat(c):
#     return LF_c_treat_d_wide(c) * cand_in_ctd_therapy(c)

# def LF_ctd_unspecified_treat(c):
#     return LF_c_treat_d_wide(c) * cand_in_ctd_unspecified(c)

# def LF_ctd_unspecified_induce(c):
#     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_unspecified(c)

def LF_closer_chem(c):
    # Get distance between chemical and disease
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    # Try to find chemical closer than @dist/2 in either direction
    sent = c.get_parent()
    closest_other_chem = float('inf')
    for i in range(dis_end, min(len(sent.words), dis_end + dist // 2)):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
            return (-1,1)
    for i in range(max(0, dis_start - dist // 2), dis_start):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
            return (-1,1)
    return (0,0)

def LF_closer_dis(c):
    # Get distance between chemical and disease
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    # Try to find chemical disease than @dist/8 in either direction
    sent = c.get_parent()
    for i in range(chem_end, min(len(sent.words), chem_end + dist // 8)):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Disease' and cid != sent.entity_cids[dis_start]:
            return (-1,1)
    for i in range(max(0, chem_start - dist // 8), chem_start):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Disease' and cid != sent.entity_cids[dis_start]:
            return (-1,1)
    return (0,0)


LFs = [LF_c_cause_d,LF_c_d,LF_c_induced_d,LF_c_treat_d,LF_c_treat_d_wide,LF_closer_chem,
    LF_closer_dis,LF_ctd_marker_c_d,LF_ctd_marker_induce,LF_ctd_therapy_treat,
    LF_ctd_unspecified_treat,LF_ctd_unspecified_induce,LF_d_following_c,
    LF_d_induced_by_c,LF_d_induced_by_c_tight,LF_d_treat_c,LF_develop_d_following_c,
    LF_far_c_d,LF_far_d_c,LF_improve_before_disease,LF_in_ctd_therapy,
    LF_in_ctd_marker,LF_in_patient_with,LF_induce,LF_induce_name,LF_induced_other,
    LF_level,LF_measure,LF_neg_d,LF_risk_d,LF_treat_d,LF_uncertain,LF_weak_assertions
]

LF_l = [
    1,1,1,-1,-1,-1,
    -1,1,1,-1,
    -1,1,1,
    1,1,-1,1,
    -1,-1,-1,-1,
    1,-1,1,1,-1,
    -1,-1,-1,1,-1,-1,-1
]
print(len(LFs),len(LF_l))

33 33


In [5]:
import numpy as np

def distanceCD(c):
    dist = 0
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    return dist/5000


def distanceCD_(c,l):
    dist = []
    for w in l:
        pattern = r'({{A}})(.*)('+w+r')(.*)({{B}})'
        matchObj = re.search(pattern, get_tagged_text(c), flags=re.I)
        if(matchObj):
            match_groups = matchObj.group(2,4)
            dist.append(sum([len(mg) for mg in match_groups]))
    if(len(dist)>0):
        return min(dist)
    return 0

def distanceDC_(c,l):
    dist = []
    for w in l:
        pattern = r'({{B}})(.*)('+w+r')(.*)({{A}})'
        matchObj = re.search(pattern, get_tagged_text(c), flags=re.I)
        if(matchObj):
            match_groups = matchObj.group(2,4)
            dist.append(sum([len(mg) for mg in match_groups]))
    if(len(dist)>0):
        return min(dist)
    return 0

   

def levenshtein(source, target):
    if len(source) < len(target):
        return levenshtein(target, source)

    # So now we have len(source) >= len(target).
    if len(target) == 0:
        return len(source)

    # We call tuple() to force strings to be used as sequences
    # ('c', 'a', 't', 's') - numpy uses them as values by default.
    source = np.array(tuple(source))
    target = np.array(tuple(target))

    # We use a dynamic programming algorithm, but with the
    # added optimization that we only need the last two rows
    # of the matrix.
    previous_row = np.arange(target.size + 1)
    for s in source:
        # Insertion (target grows longer than source):
        current_row = previous_row + 1

        # Substitution or matching:
        # Target and source items are aligned, and either
        # are different (cost of 1), or are the same (cost of 0).
        current_row[1:] = np.minimum(
                current_row[1:],
                np.add(previous_row[:-1], target != s))

        # Deletion (target grows shorter than source):
        current_row[1:] = np.minimum(
                current_row[1:],
                current_row[0:-1] + 1)

        previous_row = current_row

    return previous_row[-1]

In [7]:
# ##### Smooth LFs #########

# def cand_in_ctd_unspecified(c):
#     return 1 if c.get_cids() in ctd_unspecified else 0

# def cand_in_ctd_therapy(c):
#     return 1 if c.get_cids() in ctd_therapy else 0

# def cand_in_ctd_marker(c):
#     return 1 if c.get_cids() in ctd_marker else 0


# def LF_in_ctd_unspecified(c):
#     if(cand_in_ctd_unspecified(c)==1):
#         return (-1,1)
#     else:
#         return (0,0)

# def LF_in_ctd_therapy(c):
#     if(cand_in_ctd_therapy(c)==1):
#         return (-1,1)
#     else:
#         return (0,0)

# def LF_in_ctd_marker(c):
#     if(cand_in_ctd_marker(c)==1):
#         return (1,1)
#     else:
#         return (0,0)

# import re
# from snorkel.lf_helpers import (
#     get_tagged_text,
#     rule_regex_search_tagged_text,
#     rule_regex_search_btw_AB,
#     rule_regex_search_btw_BA,
#     rule_regex_search_before_A,
#     rule_regex_search_before_B,
# )

# import re
# from snorkel.lf_helpers import (
#     get_left_tokens, get_right_tokens, get_between_tokens,
#     get_text_between, get_tagged_text,
# )

# # List to parenthetical
# def ltp(x):
#     return '(' + '|'.join(x) + ')'

# # def LF_induce(c):
# #     return (1,1) if re.search(r'{{A}}.{0,20}induc.{0,20}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

# def LF_induce(c):
#     return (1,distanceCD_(c,['induc'])) if re.search(r'{{A}}.*induc.*{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

# causal_past = ['induced', 'caused', 'due']
# # def LF_d_induced_by_c(c):
# #     if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + '.{0,9}(by|to).{0,50}', 1)==1):
# #         return (1,1)
# #     return (0,0)

# def LF_d_induced_by_c(c):
#     sc = 0
#     word_vectors = get_word_vectors(get_between_tokens(c))
#     for w in causal_past:
#         sc=max(sc,get_similarity(word_vectors,w))
#     return (1,sc)

# # def LF_d_induced_by_c_tight(c):
# #     if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + ' (by|to) ', 1)==1):
# #         return (1,1)
# #     return (0,0)

# def LF_d_induced_by_c_tight(c):
#     if (rule_regex_search_btw_BA(c, '.*' + ltp(causal_past) + ' (by|to) ', 1)==1):
#         return (1,(1-distanceDC_(c,causal_past)))
#     return (0,0)

# def LF_induce_name(c):
#     return (1,1) if 'induc' in c.chemical.get_span().lower() else (0,0)     

# causal = ['cause[sd]?', 'induce[sd]?', 'associated with']
# # def LF_c_cause_d(c):
# #     return (1,1) if (
# #         re.search(r'{{A}}.{0,50} ' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
# #         and not re.search('{{A}}.{0,50}(not|no).{0,20}' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
# #     ) else (0,0)


# def LF_c_cause_d(c):
#     return (1,(1-distanceCD_(c,causal))) if (
#         re.search(r'{{A}}.* ' + ltp(causal) + '.*{{B}}', get_tagged_text(c), re.I)
#         and not re.search('{{A}}.{0,50}(not|no).{0,20}' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
#     ) else (0,0)


# treat = ['treat', 'effective', 'prevent', 'resistant', 'slow', 'promise', 'therap']
# # def LF_d_treat_c(c):
# #     if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_d_treat_c(c):
#     if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
#         return (-1,1-distanceDC_(c,treat))
#     return (0,0)


# # def LF_c_treat_d(c):
# #     if (rule_regex_search_btw_AB(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_c_treat_d(c):
#     if (rule_regex_search_btw_AB(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
#         return (-1,1-distanceCD_(c,treat))
#     return (0,0)

# # def LF_treat_d(c):
# #     if (rule_regex_search_before_B(c, ltp(treat) + '.{0,50}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_treat_d(c):
#     sc = 0
#     word_vectors = get_word_vectors(get_left_tokens(c[1],7))
#     for w in treat:
#         sc=max(sc,get_similarity(word_vectors,w))
#     if(re.search('(not|no|none) .* {{B}}', get_tagged_text(c), re.I)):
#         return (0,0)
#     else:
#         return (-1,sc)

# # def LF_c_treat_d_wide(c):
# #     if (rule_regex_search_btw_AB(c, '.{0,200}' + ltp(treat) + '.{0,200}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_c_treat_d_wide(c):
#     if (rule_regex_search_btw_AB(c, '.{0,200}' + ltp(treat) + '.{0,200}', -1)==-1):
#         return (-1,1-distanceCD_(c,treat))
#     return (0,0)

# def LF_c_d(c):
#     return (1,1) if ('{{A}} {{B}}' in get_tagged_text(c)) else (0,0)

# def LF_c_induced_d(c):
#     return (1,1) if (
#         ('{{A}} {{B}}' in get_tagged_text(c)) and 
#         (('-induc' in c[0].get_span().lower()) or ('-assoc' in c[0].get_span().lower()))
#         ) else (0,0)

# # def LF_improve_before_disease(c):
# #     if(rule_regex_search_before_B(c, 'improv.*', -1)==-1):
# #         return (-1,1)
# #     return (0,0)


# def distanceImproveBeforeDisease(c):
#     m=re.search(r'(improv)(.*)({{B}})', get_tagged_text(c), flags=re.I)
#     if(m):
#         return len(m.group(2))/5000
#     return 0


# def LF_improve_before_disease(c):
#     if(rule_regex_search_before_B(c, 'improv.*', -1) == -1):
#         return (-1,1-distanceImproveBeforeDisease(c))
#     else:
#         return (0,0)


# pat_terms = ['in a patient with ', 'in patients with']
# def LF_in_patient_with(c):
#     return (-1,1) if re.search(ltp(pat_terms) + '{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

# uncertain = ['combin', 'possible', 'unlikely']
# # def LF_uncertain(c):
# #     if (rule_regex_search_before_A(c, ltp(uncertain) + '.*', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_uncertain(c):
#     sc = 0
#     word_vectors = get_word_vectors(get_left_tokens(c[1],7))
#     for w in uncertain:
#         sc=max(sc,get_similarity(word_vectors,w))
#     if(re.search('(not|no|none) .* {{B}}', get_tagged_text(c), re.I)):
#         return (0,0)
#     else:
#         return (-1,sc)

# # def LF_induced_other(c):
# #     if (rule_regex_search_tagged_text(c, '{{A}}.{20,1000}-induced {{B}}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)  

# def LF_induced_other(c):
#     if (rule_regex_search_tagged_text(c, '{{A}}.{20,1000}-induced {{B}}', -1)==-1):
#         return (-1,distanceCD(c))
#     return (0,0)  

# # def LF_far_c_d(c):
# #     if (rule_regex_search_btw_AB(c, '.{100,5000}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_far_c_d(c):
#     if (rule_regex_search_btw_AB(c, '.{100,5000}', -1)==-1):
#         return (-1,distanceCD(c))
#     return (0,0)

# # def LF_far_d_c(c):
# #     if (rule_regex_search_btw_BA(c, '.{100,5000}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_far_d_c(c):
#     if (rule_regex_search_btw_BA(c, '.{100,5000}', -1)==-1):
#         return (-1,distanceCD(c))
#     return (0,0)


# # def LF_risk_d(c):
# #     if (rule_regex_search_before_B(c, 'risk of ', 1)==1):
# #         return (1,1)
# #     return (0,0)


# def LF_risk_d(c):
#     sc = 0
#     word_vectors = get_word_vectors(get_left_tokens(c[1],7))
#     sc=max(sc,get_similarity(word_vectors,'risk'))
#     return (1,sc)


# # def LF_develop_d_following_c(c):
# #     return (1,1) if re.search(r'develop.{0,25}{{B}}.{0,25}following.{0,25}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceDevFol(c):
#     dist = 0
#     matchObj = re.search(r'(develop)(.*)({{B}})(.*)(following)(.*)({{A}})', get_tagged_text(c), flags=re.I)
#     if(matchObj):
#         match_groups = matchObj.group(2,4,6)
#         dist = sum([len(mg) for mg in match_groups])
#     return dist/5000

# def LF_develop_d_following_c(c):
#     return (1,1-distanceDevFol(c)) if re.search(r'develop.*{{B}}.*following.*{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


# procedure, following = ['inject', 'administrat'], ['following']
# # def LF_d_following_c(c):
# #     return (1,distanceDFollC(c)) if re.search('{{B}}.{0,50}' + ltp(following) + '.{0,20}{{A}}.{0,50}' + ltp(procedure), get_tagged_text(c), flags=re.I) else (0,0)


# def LF_d_following_c(c):
#     return (1,1-distanceDC_(c,following)) if re.search('{{B}}.*' + ltp(following) + '.*{{A}}.*' + ltp(procedure), get_tagged_text(c), flags=re.I) else (0,0)

# # def LF_measure(c):
# #     return (-1,1) if re.search('measur.{0,75}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceMeasureA(c):
#     m = re.search('(measur)(.*)({{A}})', get_tagged_text(c), flags=re.I) 
#     if(m):
#         return (5000-len(m.group(2)))/5000
#     return 0

# def LF_measure(c):
#     return (-1,distanceMeasureA(c)) if re.search('measur.{0,75}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


# # def LF_level(c):
# #     return (-1,1) if re.search('{{A}}.{0,25} level', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceLevel(c):
#     m = re.search('({{A}})(.*)(level)', get_tagged_text(c), flags=re.I)
#     if(m):
#         return (5000-len(m.group(2)))/5000
#     return 0

# def LF_level(c):
#     return (-1,distanceLevel(c)) if re.search('{{A}}.{0,25} level', get_tagged_text(c), flags=re.I) else (0,0)

# # def LF_neg_d(c):
# #     return (-1,1) if re.search('(none|not|no) .{0,25}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceNeg(c):
#     m = re.search('(none|not|no)(.*)({{B}})', get_tagged_text(c), flags=re.I)
#     if(m):
#         return (5000-len(m.group(2)))/5000
#     return 0


# def LF_neg_d(c):
#     return (-1,distanceNeg(c)) if re.search('(none|not|no) .{0,25}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)


# WEAK_PHRASES = ['none', 'although', 'was carried out', 'was conducted',
#                 'seems', 'suggests', 'risk', 'implicated',
#                'the aim', 'to (investigate|assess|study)']

# WEAK_RGX = r'|'.join(WEAK_PHRASES)

# def LF_weak_assertions(c):
#     return (-1,1) if re.search(WEAK_RGX, get_tagged_text(c), flags=re.I) else (0,0)


# def LF_ctd_marker_c_d(c):
#     l,s = LF_c_d(c)
#     cl = cand_in_ctd_marker(c)
#     return (l*cl,s*cl)

# def LF_ctd_marker_induce(c):
#     l1,s1 = LF_c_induced_d(c)
#     l2,s2 = LF_d_induced_by_c_tight(c)
#     cl = cand_in_ctd_marker(c)
#     return ((l1 or l2)*cl,max(s1,s2)*cl)

# def LF_ctd_therapy_treat(c):
#     l,s = LF_c_treat_d_wide(c)
#     cl = cand_in_ctd_therapy(c)
#     return (l*cl,s*cl)

# def LF_ctd_unspecified_treat(c):
#     l,s = LF_c_treat_d_wide(c)
#     cl = cand_in_ctd_unspecified(c)
#     return (l*cl,s*cl)

# def LF_ctd_unspecified_induce(c):
#     l1,s1 = LF_c_induced_d(c)
#     l2,s2 = LF_d_induced_by_c_tight(c)
#     cl = cand_in_ctd_unspecified(c)
#     return ((l1 or l2)*cl,max(s1,s2)*cl)


# # def LF_ctd_marker_c_d(c):
# #     return LF_c_d(c) * cand_in_ctd_marker(c)

# # def LF_ctd_marker_induce(c):
# #     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_marker(c)

# # def LF_ctd_therapy_treat(c):
# #     return LF_c_treat_d_wide(c) * cand_in_ctd_therapy(c)

# # def LF_ctd_unspecified_treat(c):
# #     return LF_c_treat_d_wide(c) * cand_in_ctd_unspecified(c)

# # def LF_ctd_unspecified_induce(c):
# #     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_unspecified(c)

# def LF_closer_chem(c):
#     # Get distance between chemical and disease
#     chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
#     dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
#     if dis_start < chem_start:
#         dist = chem_start - dis_end
#     else:
#         dist = dis_start - chem_end
#     # Try to find chemical closer than @dist/2 in either direction
#     sent = c.get_parent()
#     closest_other_chem = float('inf')
#     for i in range(dis_end, min(len(sent.words), dis_end + dist // 2)):
#         et, cid = sent.entity_types[i], sent.entity_cids[i]
#         if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
#             return (-1,1)
#     for i in range(max(0, dis_start - dist // 2), dis_start):
#         et, cid = sent.entity_types[i], sent.entity_cids[i]
#         if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
#             return (-1,1)
#     return (0,0)

# def LF_closer_dis(c):
#     # Get distance between chemical and disease
#     chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
#     dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
#     if dis_start < chem_start:
#         dist = chem_start - dis_end
#     else:
#         dist = dis_start - chem_end
#     # Try to find chemical disease than @dist/8 in either direction
#     sent = c.get_parent()
#     for i in range(chem_end, min(len(sent.words), chem_end + dist // 8)):
#         et, cid = sent.entity_types[i], sent.entity_cids[i]
#         if et == 'Disease' and cid != sent.entity_cids[dis_start]:
#             return (-1,1)
#     for i in range(max(0, chem_start - dist // 8), chem_start):
#         et, cid = sent.entity_types[i], sent.entity_cids[i]
#         if et == 'Disease' and cid != sent.entity_cids[dis_start]:
#             return (-1,1)
#     return (0,0)


# LFs = [LF_c_cause_d,LF_c_d,LF_c_induced_d,LF_c_treat_d,LF_c_treat_d_wide,LF_closer_chem,
#     LF_closer_dis,LF_ctd_marker_c_d,LF_ctd_marker_induce,LF_ctd_therapy_treat,
#     LF_ctd_unspecified_treat,LF_ctd_unspecified_induce,LF_d_following_c,
#     LF_d_induced_by_c,LF_d_induced_by_c_tight,LF_d_treat_c,LF_develop_d_following_c,
#     LF_far_c_d,LF_far_d_c,LF_improve_before_disease,LF_in_ctd_therapy,
#     LF_in_ctd_marker,LF_in_patient_with,LF_induce,LF_induce_name,LF_induced_other,
#     LF_level,LF_measure,LF_neg_d,LF_risk_d,LF_treat_d,LF_uncertain,LF_weak_assertions
# ]

# LF_l = [
#     1,1,1,-1,-1,-1,
#     -1,1,1,-1,
#     -1,1,1,
#     1,1,-1,1,
#     -1,-1,-1,-1,
#     1,-1,1,1,-1,
#     -1,-1,-1,1,-1,-1,-1
# ]
# print(len(LFs),len(LF_l))

33 33


In [70]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for i,ci in enumerate(cands):
        L_S_ci=[]
        L=[]
        S=[]
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
    return L_S


In [71]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

# dev_L_S = get_L_S_Tensor(dev_cands)

# np.save("dev_L_S_smooth",np.array(dev_L_S))

# train_L_S = get_L_S_Tensor(train_cands)
# np.save("train_L_S_smooth",np.array(train_L_S))

# test_L_S = get_L_S_Tensor(test_cands)
# np.save("test_L_S_smooth",np.array(test_L_S))
test_L_S = get_L_S_Tensor(test_cands)
np.save("test_L_S_discrete",np.array(test_L_S))


print("--- %s seconds ---" % (time.time() - start_time))

# test_L_S = get_L_S_Tensor(test_cands)
# pkl.dump(test_L_S,open("test_L_S.p","wb"))

started at: 13-6-2018, 22:11:13
500data points labelled in 0.749686332543691 mins
1000data points labelled in 1.1264859318733216 mins
1500data points labelled in 1.439268696308136 mins
2000data points labelled in 1.6602157950401306 mins
2500data points labelled in 1.8612461010615031 mins
3000data points labelled in 2.0292420864105223 mins
3500data points labelled in 2.182789929707845 mins
4000data points labelled in 2.3764989376068115 mins
4500data points labelled in 2.5716164191563924 mins
--- 156.4515299797058 seconds ---


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(gold_labels_dev,pl))
#     print(precision_recall_fscore_support(true_labels,pl,average='macro'))
    print(confusion_matrix(gold_labels_dev,pl))
#     draw2DArray(confusion_matrix(gold_labels_dev,pl))
    return report2dict(classification_report(gold_labels_dev, pl))# target_names=class_names))
    


def drawPRcurve(y_test,y_score,it_no):
    
    fig = plt.figure()
    splt = fig.add_subplot(111)
    

    precision, recall, _ = precision_recall_curve(y_test, y_score,pos_label=1)

    splt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    splt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    average_precision = average_precision_score(y_test, y_score)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.05])
    plt.title('{0:d} Precision-Recall curve: AP={1:0.2f}'.format(it_no,
              average_precision))
  

In [59]:
LF_l = [
    1,1,1,-1,-1,-1,
    -1,1,1,-1,
    -1,1,1,
    1,1,-1,1,
    -1,-1,-1,-1,
    1,-1,1,1,-1,
    -1,-1,-1,1,-1,-1,-1
]

import numpy as np
dev_L_S = np.load("dev_L_S_smooth.npy")
train_L_S = np.load("train_L_S_smooth.npy")
test_L_S = np.load("test_L_S_smooth.npy")

# dev_L_S = np.load("dev_L_S_discrete.npy")
# train_L_S = np.load("train_L_S_discrete.npy")

test_L_S = dev_L_S
# true_labels = gold_labels_test
true_labels = gold_labels_dev
print(len(true_labels))
print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)

888
(888, 2, 33) (888, 2, 33) (8272, 2, 33)


In [ ]:
LF_l = [
    1,1,1,-1,-1,-1,
    -1,1,1,-1,
    -1,1,1,
    1,1,-1,1,
    -1,-1,-1,-1,
    1,-1,1,1,-1,
    -1,-1,-1,1,-1,-1,-1
]


def merge(a,b):
    c = []
    for i in range(len(a)):
        ci = []
        ci_l = a[i,0,:].tolist()+b[i,0,:].tolist()
        ci_s = a[i,1,:].tolist()+b[i,1,:].tolist()
        ci.append(ci_l)
        ci.append(ci_s)
        c.append(ci)
    return c
import numpy as np
dev_L_S_s = np.load("dev_L_S_smooth.npy")
train_L_S_s = np.load("train_L_S_smooth.npy")

dev_L_S_d = np.load("dev_L_S_discrete.npy")
train_L_S_d = np.load("train_L_S_discrete.npy")

dev_L_S = np.array(merge(dev_L_S_d,dev_L_S_s))
train_L_S = np.array(merge(train_L_S_d,train_L_S_s))

LF_l = LF_l + LF_l
print(len(LF_l))
test_L_S = dev_L_S
true_labels = gold_labels_dev
print(dev_L_S.shape,train_L_S.shape)

In [30]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32
seed = 12

In [31]:
NoOfLFs= len(LF_l)
NoOfClasses = 2
print(len(LF_l))

33


In [62]:
## normalized model with smooth LFs + penalties

def train(lr,ep,th,af,pcl=np.array([-1,1],dtype=np.float64),norm=True,smooth=True,penalty=0,p3k=3,alp=0.99):
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## pcl : all possible class labels  = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    ## alp : alpha parameter (don't let alpha to be greater than this value)
    BATCH_SIZE = 1
    tf.reset_default_graph()
    

    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
#         print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
        print(s)
        print("l",l)
#         print(s.graph)
        
        s_ = tf.maximum(tf.subtract(s,tf.minimum(alphas,alp)), 0)
        print("s_",s_)

       
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
            print("out",out)
            return out

        if(smooth):
            pout = tf.map_fn(lambda c: l*c*s_ ,pcl,name="pout")
        else:
            pout = tf.map_fn(lambda c: l*c ,pcl,name="pout")

        print("pout",pout)    

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                           name="t_pout")
    
        print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
            print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
            print("intsy",out1)
            return out1
                

        if(smooth):
            #smooth normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="zy")
        else:
            #discrete normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="zy")

    
# 
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        
        print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        
        print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
        print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))
        
        if(not norm):
            print("unnormlized loss")
            loss = tf.negative(tf.reduce_sum(lsp  ))
        elif(penalty == 1):
            print("penalty1")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas))
        elif(penalty == 2):
            print("penalty2")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -tf.minimum( tf.reduce_min(thetas),0.0)
        elif(penalty == 3):
            print("penalty3")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k)))
        else:
            loss = tf.negative(tf.reduce_sum(lsp  - logz  ))
            
        print("loss",loss)
        tf.summary.scalar('un-normloss', loss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(loss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,loss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)

                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
                print(a)
                print(t)
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(true_labels,pl))
                print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(true_labels,pl))

            print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [60]:
# dev data
for i in np.linspace(0,1,11):
    print()
    print("alpha-mean",i)
    train(0.1/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(i,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),norm=False,smooth=True,penalty=0)


alpha-mean 0.0
Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
unnormlized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss -50601.36766025227
[ 1.17544629e-03 -3.21950217e-02 -2.61857480e-02  6.24306244e-06


1 loss -52707.29595333156
[ 0.10117545  0.03739454  0.04733319  0.10000624  0.10106378  0.11647109
  0.10000329  0.04194336  0.0428765   0.10064723  0.09983571  0.04823313
  0.09982382 -0.03035453  0.10040812  0.10042236  0.09161212  0.12710726
  0.10023417  0.09661885  0.12357171 -0.04200104  0.09952227  0.03872272
  0.03336547  0.14238716  0.09578464  0.1032263   0.11705497 -0.03131544
  0.22005303  0.2202263   0.11503341]
[[1.11754463 0.87494032 1.06233906 1.00062431 1.10637787 1.02528576
  1.1708953  1.08960074 1.20305083 1.0647233  0.98357064 1.11747512
  0.98238184 1.20728547 1.04081247 1.04223612 0.86026449 0.89209351
  1.06002404 0.83816347 1.01216759 1.08289422 0.99665456 1.20762395
  0.93909795 1.13701729 1.07299058 1.06596871 1.04100254 1.12081509
  0.70409375 0.71901413 0.94674021]]
{0: 85, 1: 803}
acc 0.4155405405405405
(0.36114570361145704, 0.9797297297297297, 0.5277525022747953, None)

2 loss -57498.372164115404
[ 0.10117545  0.00694109  0.02085468  0.10000624  0.1010637

4 loss -69164.2816419689
[ 0.20117545  0.0460992   0.06790215  0.20000624  0.20106378  0.33831112
  0.23359742  0.05434912  0.05463149  0.20064723  0.19983571  0.06880219
  0.19982382 -0.22869752  0.20040812  0.20042236  0.18129427  0.32244973
  0.282208    0.20347984  0.37884869 -0.17924123  0.20754852  0.04520117
  0.02952488  0.30259158  0.21469153  0.21823429  0.25489969 -0.22985689
  0.59805228  0.59828348  0.30662304]
[[1.11754463 0.9662658  1.14184275 1.00062431 1.10637787 0.90364617
  1.13731802 1.17728736 1.2879637  1.0647233  0.98357064 1.19388
  0.98238184 1.50325821 1.04081247 1.04223612 0.87058865 0.79793725
  0.97874603 0.83130473 0.85723384 1.32050422 0.98862875 1.27277505
  1.04300134 1.07738926 1.05408411 1.05096235 1.00316801 1.42003827
  0.44387826 0.45532269 0.85522994]]
{1: 888}
acc 0.3333333333333333
(0.3333333333333333, 1.0, 0.5, None)

[ 0.20117545  0.0460992   0.06790215  0.20000624  0.20106378  0.33831112
  0.23359742  0.05434912  0.05463149  0.20064723  0.199

0 loss -46757.766076936954
[0.40117545 0.36803725 0.37381145 0.40000624 0.40106378 0.40724562
 0.40058015 0.37118376 0.37241012 0.40064723 0.39983571 0.37471705
 0.39982382 0.34253092 0.40040812 0.40042236 0.39506042 0.41386713
 0.4017082  0.39687027 0.40462546 0.33241291 0.39910017 0.36989929
 0.36687531 0.42231825 0.39691195 0.40107624 0.40608174 0.34161231
 0.4524587  0.45261549 0.40378313]
[[1.11754463 0.844355   1.03588696 1.00062431 1.10637787 1.03451293
  1.17031849 1.06039    1.17314342 1.0647233  0.98357064 1.09070488
  0.98238184 1.13313375 1.04081247 1.04223612 0.85681548 0.90547294
  1.05858115 0.8379096  1.03110902 1.00866456 0.99707673 1.18579864
  0.90564287 1.15733283 1.0718608  1.06811893 1.05197615 1.04730006
  0.77347603 0.78779557 0.95799055]]
{0: 203, 1: 685}
acc 0.4988738738738739
(0.39124087591240875, 0.9054054054054054, 0.54638124362895, None)

1 loss -49506.77355961854
[0.40117545 0.33762439 0.34732579 0.40000624 0.40106378 0.43044672
 0.4056183  0.34194664 0.3

3 loss -56986.23671285713
[0.50117545 0.37689497 0.39435474 0.50000624 0.50106378 0.61763441
 0.53086626 0.38354434 0.38438103 0.50064723 0.49983571 0.39554548
 0.49982382 0.15899209 0.50040812 0.50042236 0.48468777 0.51125524
 0.50981186 0.50233008 0.64212703 0.19763101 0.50527845 0.37637442
 0.36381396 0.50404395 0.51145571 0.51425386 0.53932219 0.15788261
 0.75900201 0.76537672 0.5868275 ]
[[1.11754463 0.93572405 1.11557411 1.00062431 1.10637787 0.92473234
  1.14007406 1.1483107  1.2528789  1.0647233  0.98357064 1.16247316
  0.98238184 1.40432207 1.04081247 1.04223612 0.86719963 0.91163618
  1.05372668 0.83245631 0.89459682 1.24480034 0.99089965 1.25070771
  1.00901079 1.17585046 1.05732084 1.0549449  1.01877604 1.3305244
  0.61216383 0.61821274 0.87523796]]
{1: 888}
acc 0.3333333333333333
(0.3333333333333333, 1.0, 0.5, None)

4 loss -62100.94968429492
[0.50117545 0.34644955 0.36787953 0.50000624 0.50106378 0.66126891
 0.54369206 0.35434858 0.35496429 0.50064723 0.49983571 0.3690639

marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss -44242.36908317181
[0.70117545 0.66850369 0.67378882 0.70000624 0.70106378 0.7112175
 0.70123985 0.67144744 0.67566851 0.70064723 0.69983571 0.67739796
 0.69982382 0.66292959 0.70040812 0.70042236 0.69509273 0.69919935
 0.70059625 0.6967595  0.70394364 0.64117199 0.69865773 0.66990011
 0.6684367  0.70177478 0.69565543 0.70079033 0.69995209 0.66818025
 0.72427658 0.73313575 0.70373266]
[[1.11754463 0.84401484 1.03601242 1.00062431 1.10637787 1.03056981
  1.16965975 1.06024752 1.17227822 1.0647233  0.98357064 1.08999992
  0.98238184 1.11551174 1.04081247 1.04223612 0.85678544 0.91993454
  1.05962482 0.83801966 1.03179412 1.00030506 0.99751966 1.18571692
  0.90420951 1.17747756 1.07312139 1.06840598 1.05809933 1.01532956
  0.81386432 0.81314108 0.95804423]]
{0: 220, 1: 668}
acc 0.5180180180180181
(0.40119760479041916, 0.9054054054054054, 0.5560165975103735, None)

1 loss -45595.89639382747
[0.70117545 0.63840424 0.64

3 loss -47205.03868058718
[0.80117545 0.67894603 0.6941208  0.80000624 0.80106378 0.8858034
 0.81581588 0.68421452 0.69751299 0.80064723 0.79983571 0.70686739
 0.79982382 0.59822665 0.80040812 0.80042236 0.78460352 0.79919935
 0.80059625 0.79647939 0.88217053 0.54084265 0.7981654  0.67637458
 0.67305395 0.80177478 0.79375022 0.80766723 0.811905   0.75350498
 0.83319659 0.82453525 0.85416605]
[[1.11754463 0.93448591 1.11649038 1.00062431 1.10637787 0.95727857
  1.15513293 1.14843179 1.2512485  1.0647233  0.98357064 1.16119596
  0.98238184 1.24096363 1.04081247 1.04223612 0.86730531 0.91993454
  1.05962482 0.83829987 0.95499224 1.20403773 0.99801193 1.25037748
  1.00062238 1.17747756 1.07505785 1.06153382 1.04618891 1.03286942
  0.80603262 0.82010505 0.90810985]]
{0: 43, 1: 845}
acc 0.38175675675675674
(0.3502958579881657, 1.0, 0.518843120070114, None)

4 loss -49041.29093851959
[0.80117545 0.64883733 0.66763092 0.80000624 0.80106378 0.9228569
 0.82650508 0.6549885  0.67155354 0.80064723

0 loss -42803.68162706099
[1.00117545 0.99814215 1.0000956  1.00000624 1.00106378 1.00041339
 1.00169206 1.00031208 1.00146335 1.00064723 0.99983571 1.00066614
 0.99982382 1.00078582 1.00040812 1.00042236 0.99853341 0.99919935
 1.00059625 0.99836416 1.00035376 0.99941435 0.99996215 1.00163011
 0.99873733 1.00177478 1.00068086 1.00068508 1.00057476 0.99990344
 0.99821992 0.99838377 0.99962151]
[[1.11754463 0.84251149 1.03554932 1.00062431 1.10637787 1.01837111
  1.16735218 1.05895929 1.17192859 1.0647233  0.98357064 1.08971618
  0.98238184 1.09720511 1.04081247 1.04223612 0.85611545 0.91993454
  1.05962482 0.83789125 1.01821328 0.96816888 0.9977009  1.18563011
  0.887074   1.17747756 1.07030433 1.06800898 1.05633042 0.99728524
  0.82110581 0.83819336 0.95230561]]
{0: 455, 1: 433}
acc 0.6835585585585585
(0.5173210161662818, 0.7567567567567568, 0.6145404663923183, None)

1 loss -43493.872446998335
[1.00117545 0.99814215 1.0000956  1.00000624 1.00106378 1.00041339
 1.00169206 1.00031208 1.

In [57]:
# test data
for i in np.linspace(0,1,11):
    print()
    print("alpha-mean",i)
    train(0.1/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(i,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),norm=False,smooth=True,penalty=0)


alpha-mean 0.0
Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
unnormlized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss -50601.36766025227
[ 1.17544629e-03 -3.21950217e-02 -2.61857480e-02  6.24306244e-06


1 loss -52707.29595333156
[ 0.10117545  0.03739454  0.04733319  0.10000624  0.10106378  0.11647109
  0.10000329  0.04194336  0.0428765   0.10064723  0.09983571  0.04823313
  0.09982382 -0.03035453  0.10040812  0.10042236  0.09161212  0.12710726
  0.10023417  0.09661885  0.12357171 -0.04200104  0.09952227  0.03872272
  0.03336547  0.14238716  0.09578464  0.1032263   0.11705497 -0.03131544
  0.22005303  0.2202263   0.11503341]
[[1.11754463 0.87494032 1.06233906 1.00062431 1.10637787 1.02528576
  1.1708953  1.08960074 1.20305083 1.0647233  0.98357064 1.11747512
  0.98238184 1.20728547 1.04081247 1.04223612 0.86026449 0.89209351
  1.06002404 0.83816347 1.01216759 1.08289422 0.99665456 1.20762395
  0.93909795 1.13701729 1.07299058 1.06596871 1.04100254 1.12081509
  0.70409375 0.71901413 0.94674021]]
{0: 392, 1: 4228}
acc 0.404978354978355
(0.35288552507095555, 0.9913621262458472, 0.5204953776382348, None)

2 loss -57498.372164115404
[ 0.10117545  0.00694109  0.02085468  0.10000624  0.101063

4 loss -69164.2816419689
[ 0.20117545  0.0460992   0.06790215  0.20000624  0.20106378  0.33831112
  0.23359742  0.05434912  0.05463149  0.20064723  0.19983571  0.06880219
  0.19982382 -0.22869752  0.20040812  0.20042236  0.18129427  0.32244973
  0.282208    0.20347984  0.37884869 -0.17924123  0.20754852  0.04520117
  0.02952488  0.30259158  0.21469153  0.21823429  0.25489969 -0.22985689
  0.59805228  0.59828348  0.30662304]
[[1.11754463 0.9662658  1.14184275 1.00062431 1.10637787 0.90364617
  1.13731802 1.17728736 1.2879637  1.0647233  0.98357064 1.19388
  0.98238184 1.50325821 1.04081247 1.04223612 0.87058865 0.79793725
  0.97874603 0.83130473 0.85723384 1.32050422 0.98862875 1.27277505
  1.04300134 1.07738926 1.05408411 1.05096235 1.00316801 1.42003827
  0.44387826 0.45532269 0.85522994]]
{0: 2, 1: 4618}
acc 0.3261904761904762
(0.32589865742745777, 1.0, 0.4915890903152049, None)

[ 0.20117545  0.0460992   0.06790215  0.20000624  0.20106378  0.33831112
  0.23359742  0.05434912  0.0546

0 loss -46757.766076936954
[0.40117545 0.36803725 0.37381145 0.40000624 0.40106378 0.40724562
 0.40058015 0.37118376 0.37241012 0.40064723 0.39983571 0.37471705
 0.39982382 0.34253092 0.40040812 0.40042236 0.39506042 0.41386713
 0.4017082  0.39687027 0.40462546 0.33241291 0.39910017 0.36989929
 0.36687531 0.42231825 0.39691195 0.40107624 0.40608174 0.34161231
 0.4524587  0.45261549 0.40378313]
[[1.11754463 0.844355   1.03588696 1.00062431 1.10637787 1.03451293
  1.17031849 1.06039    1.17314342 1.0647233  0.98357064 1.09070488
  0.98238184 1.13313375 1.04081247 1.04223612 0.85681548 0.90547294
  1.05858115 0.8379096  1.03110902 1.00866456 0.99707673 1.18579864
  0.90564287 1.15733283 1.0718608  1.06811893 1.05197615 1.04730006
  0.77347603 0.78779557 0.95799055]]
{0: 971, 1: 3649}
acc 0.5194805194805194
(0.4020279528637983, 0.9747508305647841, 0.5692665890570431, None)

1 loss -49506.77355961854
[0.40117545 0.33762439 0.34732579 0.40000624 0.40106378 0.43044672
 0.4056183  0.34194664 0

3 loss -56986.23671285713
[0.50117545 0.37689497 0.39435474 0.50000624 0.50106378 0.61763441
 0.53086626 0.38354434 0.38438103 0.50064723 0.49983571 0.39554548
 0.49982382 0.15899209 0.50040812 0.50042236 0.48468777 0.51125524
 0.50981186 0.50233008 0.64212703 0.19763101 0.50527845 0.37637442
 0.36381396 0.50404395 0.51145571 0.51425386 0.53932219 0.15788261
 0.75900201 0.76537672 0.5868275 ]
[[1.11754463 0.93572405 1.11557411 1.00062431 1.10637787 0.92473234
  1.14007406 1.1483107  1.2528789  1.0647233  0.98357064 1.16247316
  0.98238184 1.40432207 1.04081247 1.04223612 0.86719963 0.91163618
  1.05372668 0.83245631 0.89459682 1.24480034 0.99089965 1.25070771
  1.00901079 1.17585046 1.05732084 1.0549449  1.01877604 1.3305244
  0.61216383 0.61821274 0.87523796]]
{0: 2, 1: 4618}
acc 0.3261904761904762
(0.32589865742745777, 1.0, 0.4915890903152049, None)

4 loss -62100.94968429492
[0.50117545 0.34644955 0.36787953 0.50000624 0.50106378 0.66126891
 0.54369206 0.35434858 0.35496429 0.500647

0 loss -44242.36908317181
[0.70117545 0.66850369 0.67378882 0.70000624 0.70106378 0.7112175
 0.70123985 0.67144744 0.67566851 0.70064723 0.69983571 0.67739796
 0.69982382 0.66292959 0.70040812 0.70042236 0.69509273 0.69919935
 0.70059625 0.6967595  0.70394364 0.64117199 0.69865773 0.66990011
 0.6684367  0.70177478 0.69565543 0.70079033 0.69995209 0.66818025
 0.72427658 0.73313575 0.70373266]
[[1.11754463 0.84401484 1.03601242 1.00062431 1.10637787 1.03056981
  1.16965975 1.06024752 1.17227822 1.0647233  0.98357064 1.08999992
  0.98238184 1.11551174 1.04081247 1.04223612 0.85678544 0.91993454
  1.05962482 0.83801966 1.03179412 1.00030506 0.99751966 1.18571692
  0.90420951 1.17747756 1.07312139 1.06840598 1.05809933 1.01532956
  0.81386432 0.81314108 0.95804423]]
{0: 1076, 1: 3544}
acc 0.5383116883116883
(0.41139954853273136, 0.9687707641196013, 0.5775401069518716, None)

1 loss -45595.89639382747
[0.70117545 0.63840424 0.64726604 0.70000624 0.70106378 0.73174868
 0.70498828 0.64225082 0

3 loss -47205.03868058718
[0.80117545 0.67894603 0.6941208  0.80000624 0.80106378 0.8858034
 0.81581588 0.68421452 0.69751299 0.80064723 0.79983571 0.70686739
 0.79982382 0.59822665 0.80040812 0.80042236 0.78460352 0.79919935
 0.80059625 0.79647939 0.88217053 0.54084265 0.7981654  0.67637458
 0.67305395 0.80177478 0.79375022 0.80766723 0.811905   0.75350498
 0.83319659 0.82453525 0.85416605]
[[1.11754463 0.93448591 1.11649038 1.00062431 1.10637787 0.95727857
  1.15513293 1.14843179 1.2512485  1.0647233  0.98357064 1.16119596
  0.98238184 1.24096363 1.04081247 1.04223612 0.86730531 0.91993454
  1.05962482 0.83829987 0.95499224 1.20403773 0.99801193 1.25037748
  1.00062238 1.17747756 1.07505785 1.06153382 1.04618891 1.03286942
  0.80603262 0.82010505 0.90810985]]
{0: 222, 1: 4398}
acc 0.37294372294372297
(0.3417462482946794, 0.998671096345515, 0.5092325935964764, None)

4 loss -49041.29093851959
[0.80117545 0.64883733 0.66763092 0.80000624 0.80106378 0.9228569
 0.82650508 0.6549885  0.67

lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
unnormlized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss -42803.68162706099
[1.00117545 0.99814215 1.0000956  1.00000624 1.00106378 1.00041339
 1.00169206 1.00031208 1.00146335 1.00064723 0.99983571 1.00066614
 0.99982382 1.00078582 1.00040812 1.00042236 0.99853341 0.99919935
 1.00059625 0.99836416 1.00035376 0.99941435 0.99996215 1.00163011
 0.99873733 1.00177478 1.00068086 1.00068508 1.00057476 0.99990344
 0.99821992 0.99838377 0.99962151]
[[1.11754463 0.84251149 1.03554932 1.00062431 1.10637787 1.01837111
  1.16735218 1.05895929 1.17192859 1.0647233  0.98357064 1.08971618
  0.98238184 1.09720511 1.04081247 1.04223612 0.85611545 0.91993454
  1.05962482 0.83789125 1.01821328 0.96816888 0.9977009  1.18563011
  0.887074   1.17747756 1.07030433 1.06800898 1.05633042 0.99728524
  0.82110581 0.83819336 0.95230561]]
{0: 2322, 1: 2298}
acc 0.71796536

In [58]:
# test data                     ---Marked
for i in np.linspace(0.8,1,3):
    print()
    print("alpha-mean",i)
    train(0.1/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),
                            af = tf.truncated_normal_initializer(i,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=0)


alpha-mean 0.8
Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 151519.0951873517
[0.89860178 0.77197473 0.77783958 0.87683398 0.87902713 0.81793164
 0.82273579 0.77

2 loss 150036.45021007184
[1.         0.8176137  0.82942681 0.85790538 0.87798975 0.97002544
 0.92376871 0.82292665 0.8328323  0.87081079 0.85401542 0.83982157
 1.         0.83362802 1.         0.86656468 0.95560257 0.83735893
 0.86987242 0.91915833 0.96920457 0.73498227 0.93296642 0.81484262
 0.81584276 0.8885331  0.92723129 0.92130329 0.91895912 0.9154729
 0.91510496 0.92215737 0.94142354]
[[1.0511611  0.89934981 1.08496221 0.7275928  0.8383118  0.97488821
  1.14890997 1.11356016 1.21989592 0.79476286 0.70968713 1.13220346
  0.91486341 1.14893442 0.97378731 0.77122086 0.81785819 0.64272643
  0.78942728 0.80740614 0.96987441 1.112877   0.96074451 1.22827136
  0.96136831 0.91248243 1.04172007 1.04939797 1.04028743 0.99194017
  0.80591905 0.81500132 0.92133018]]
{0: 1536, 1: 3084}
acc 0.6482683982683982
(0.4805447470817121, 0.9847176079734219, 0.6458923512747875, None)

3 loss 149010.02480131487
[1.         0.792684   0.80820733 0.78660618 0.82294256 0.99013171
 0.93106199 0.79923076 0.

In [33]:
for i in np.linspace(0,1,11):
    print()
    print("alpha-mean",i)
    train(0.1/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(i,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=1)


alpha-mean 0.0
Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 158843.28008797162
[ 0.02561872  0.05365972 -0.03058625  0.06756809  0.08039238  0.06706934
  0.08194717 -0.02961741 -0.01135461  0.07676846  0.063

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0 loss 158419.26558881055
[0.1499137  0.15082562 0.07270765 0.16857515 0.18145911 0.16501479
 0.18204951 0.07347361 0.09159005 0.17788657 0.16454107 0.08251467
 0.1046959  0.00691102 0.12662741 0.17533871 0.14931197 0.14670861
 0.16547624 0.07781194 0.16825034 0.02066909 0.16634309 0.08947537
 0.14194594 0.17312801 0.17605769 0.17608128 0.17293748 0.00447228
 0.00884568 0.00963754 0.1528604 ]
[[ 1.003418   -0.33227888  0.56935385  0.42696152  0.94726028  0.64687218
   1.26572992  0.67774878  1.2347549   0.74117716  0.34467785  0.84379761
   0.38483983  0.9859142   0.64942893  0.63038319 -0.13462698  0.04683403
   0.7242518  -0.21668312  0.61246263  0.28385588  0.41008266  1.33801026
  -0.03505911  1.3065888   0.76291804  0.76492721  0.71525835  0.54662728
  -0.29728758 -0.21570516  0.26285448]]
{0: 3, 1: 885}
acc 0.3367117117117117
(0.3344632768361582, 1.0, 0.5012701100762066, None)

1 loss 149789.01964146135
[ 0.25117239  0.16843505  0.09786173  0.07981161  0.24610307  0.20620177
  0.

2 loss 143223.99827703982
[ 0.46072089  0.27929093  0.21645613  0.07603492  0.2712612   0.34220655
  0.36920988  0.21199373  0.25606549  0.14598428  0.06217736  0.2416887
  0.408167   -0.09090398  0.43479648  0.11484171  0.16076337  0.26858532
  0.31404406  0.34488241  0.36825565 -0.02107536  0.13560278  0.2520049
  0.2222409   0.34130882  0.28789081  0.27822236  0.29017206 -0.09560347
 -0.0702428  -0.06000989  0.28655828]
[[ 7.94645557e-01 -1.35601424e-01  5.62542831e-01  4.80491935e-01
   7.65394940e-01  5.62027858e-01  1.16881530e+00  6.75516944e-01
   1.19869893e+00  5.64957510e-01  4.76273314e-01  8.14068656e-01
   1.78494957e-01  1.17953551e+00  4.42523757e-01  4.90917532e-01
   2.04256244e-05  4.29522692e-02  6.41682958e-01 -1.71745228e-02
   5.06585978e-01  4.31235609e-01  4.27613894e-01  1.38022021e+00
   3.37106242e-02  1.17356316e+00  7.00338370e-01  6.97680397e-01
   6.65806649e-01  7.44967655e-01 -1.11833668e-01 -3.02196990e-02
   2.11241311e-01]]
{1: 888}
acc 0.3333333333

4 loss 133762.9095588707
[ 0.76405485  0.38963446  0.33876305 -0.0338205   0.1301764   0.51394123
  0.49185489  0.32865407  0.38447934  0.03506455 -0.04827279  0.37078606
  0.65608201 -0.19031188  0.73337443  0.00622732  0.34985146  0.40156668
  0.45009059  0.2969208   0.5648575  -0.05862738  0.08504983  0.37934155
  0.28797269  0.47216511  0.35193262  0.32103444  0.38090983 -0.19635323
 -0.05155111  0.077202    0.42116959]
[[ 5.95780176e-01  6.94262350e-03  5.73286868e-01  5.55951159e-01
   5.99971725e-01  4.83870668e-01  1.13261886e+00  6.91311744e-01
   1.19393267e+00  5.71341013e-01  5.53219694e-01  8.09365382e-01
   1.97426212e-05  1.37336495e+00  2.44106098e-01  5.64404219e-01
   2.70436973e-05  8.94204296e-02  6.05960079e-01  7.71437344e-02
   4.05560286e-01  5.76831863e-01  4.76675361e-01  1.42216254e+00
   8.24734160e-02  1.02857842e+00  6.87863002e-01  6.88962729e-01
   6.48644184e-01  9.44206408e-01 -3.10785412e-07  2.14776825e-06
   1.61394026e-01]]
{1: 888}
acc 0.333333333

pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 158714.45773483493
[0.6221304  0.53527072 0.48045015 0.56752696 0.58347294 0.54437105
 0.56851871 0.48031631 0.49101135 0.57965898 0.56092623 0.48708586
 0.55063716 0.41616082 0.58296411 0.57670186 0.53864919 0.50411478
 0.57261805 0.50511759 0.55138457 0.42723249 0.5623959  0.48674664
 0.52924545 0.58755554 0.571479

1 loss 152907.24504237503
[0.80451608 0.65660472 0.57006638 0.57501338 0.72826575 0.66626391
 0.6873046  0.56900072 0.59152981 0.65827296 0.55718004 0.58727459
 0.76431492 0.45286707 0.80694194 0.62396557 0.58901129 0.50283797
 0.64988438 0.68733681 0.67967573 0.46513757 0.6301752  0.57218755
 0.61950401 0.76286451 0.68495648 0.67916453 0.66353236 0.44243736
 0.45111877 0.45336717 0.65279362]
[[ 0.89718401 -0.22967618  0.58580293  0.44071792  0.86983266  0.64361125
   1.27528148  0.696071    1.2552309   0.66086667  0.41108729  0.85765962
   0.22083383  1.01691618  0.50542189  0.55275452 -0.03341114  0.19556896
   0.63574837 -0.11787357  0.60033812  0.3421216   0.41609083  1.35877775
   0.00790069  1.22099519  0.75522238  0.76405269  0.73108081  0.60270788
  -0.19164537 -0.10974208  0.25733478]]
{0: 2, 1: 886}
acc 0.3355855855855856
(0.3340857787810384, 1.0, 0.5008460236886634, None)

2 loss 147241.31650832092
[0.89458664 0.67958805 0.56240841 0.47014216 0.62747599 0.70421696
 0.7099055

3 loss 144664.45909875765
[1.         0.80157046 0.63548678 0.48580811 0.66420649 0.80602113
 0.79210633 0.63244537 0.66417163 0.57426502 0.46499335 0.66198967
 0.99999998 0.46797392 1.         0.53815478 0.72123049 0.50376205
 0.56526405 0.83333677 0.81578783 0.46397294 0.67886664 0.62787157
 0.657702   0.79489623 0.78426002 0.77926852 0.76312375 0.45246648
 0.48791626 0.55182962 0.77953434]
[[ 8.38108555e-01 -2.94620943e-02  6.28132832e-01  3.43045257e-01
   6.59976711e-01  6.02484200e-01  1.27205130e+00  7.40867571e-01
   1.29259463e+00  4.52834673e-01  3.32919975e-01  8.91992293e-01
   1.19650239e-01  1.08176349e+00  4.54319176e-01  3.72142703e-01
   2.60561328e-04  3.35625276e-01  4.29792622e-01  3.74634823e-04
   5.63356999e-01  4.46981771e-01  4.21662442e-01  1.40185305e+00
   6.16364723e-02  1.03067270e+00  7.49004692e-01  7.59632225e-01
   7.27484167e-01  6.55540575e-01 -2.82462018e-07 -7.39981762e-08
   2.26967996e-01]]
{1: 888}
acc 0.3333333333333333
(0.3333333333333333, 1.0

[1.         0.89834402 0.70189521 0.49030516 0.78781152 0.90010989
 0.85597807 0.698607   0.72651853 0.63683285 0.46778103 0.72775197
 1.         0.6421573  1.         0.55806093 0.85640121 0.50407067
 0.61565832 0.86513568 0.87421853 0.58119035 0.80587618 0.68346733
 0.73393354 0.87836729 0.86151218 0.86582353 0.83671997 0.89469759
 0.79713841 0.80037992 0.85734633]
[[9.32636226e-01 2.54392722e-03 6.59234733e-01 1.95766074e-01
  5.67211278e-01 6.08697016e-01 1.30610489e+00 7.72296151e-01
  1.32667190e+00 3.43775494e-01 1.89458060e-01 9.22642718e-01
  2.56142536e-01 1.02202089e+00 5.47745884e-01 2.37420506e-01
  3.13925228e-04 1.95554620e-01 3.17606388e-01 4.26426469e-03
  6.04361861e-01 4.30995163e-01 4.32475828e-01 1.42341429e+00
  7.89167103e-02 9.41859837e-01 7.81937122e-01 7.83160334e-01
  7.56283724e-01 4.06131440e-01 1.00588739e-05 2.96656946e-03
  2.50716308e-01]]
{0: 39, 1: 849}
acc 0.37725225225225223
acc 0.37725225225225223
[[ 39 553]
 [  0 296]]
(0.3486454652532391, 1.0, 0.

0 loss 159752.50031546722
[1.14985856 1.         1.14996198 1.         1.14795612 1.14419846
 1.1464229  1.15020164 1.14911851 1.14813356 1.         1.14979837
 0.90135902 1.14860973 1.15099026 1.14824735 1.         1.
 1.14815919 0.95440094 1.14423229 0.96710983 1.         1.14813732
 1.         1.14777165 1.14710067 1.14769953 1.1461097  0.98057584
 0.98583677 0.9729823  1.        ]
[[ 1.24391186 -0.32892728  0.62933448  0.50312153  1.18646331  0.68719111
   1.38294311  0.74141008  1.327495    0.97753591  0.41785319  0.92252819
   0.29366863  0.92001505  0.85955772  0.86460948 -0.1332933   0.09967063
   0.95194552 -0.21791993  0.66439735  0.23471164  0.48275231  1.337684
  -0.03133251  1.54277986  0.94208221  0.95537695  0.83619976  0.46191953
  -0.29004304 -0.2081127   0.30066218]]
{0: 349, 1: 539}
acc 0.6542792792792793
(0.4897959183673469, 0.8918918918918919, 0.6323353293413174, None)

1 loss 154143.7041498833
[1.26699316 1.         1.2691074  1.         1.26326831 1.25549143
 1.2

In [65]:
train(0.1/len(train_L_S),3,th = tf.truncated_normal_initializer(0.5,0.5,seed),\
                            af = tf.truncated_normal_initializer(1,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=3,alp=2)

Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
penalty3
loss Tensor("add_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 155579.5835473317
[1.14960575 0.97997786 1.15064337 1.         1.14651259 1.14607332
 1.14669631 1.1507333

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [68]:
train(0.1/len(train_L_S),6,th = tf.truncated_normal_initializer(0.5,0.5,seed),\
                            af = tf.truncated_normal_initializer(1,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=3,alp=0.95)

Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
penalty3
loss Tensor("add_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 155546.31573946652
[1.14920962 1.         1.15002906 1.         1.1472071  1.14688585
 1.14718322 1.150197

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [30]:
# train(0.1/len(train_L_S),3,th = tf.truncated_normal_initializer(0.5,0.5,seed),\
#                             af = tf.truncated_normal_initializer(1,0.001,seed),
#                           pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=2,alp=2)

Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 150433.37563495414
[1.15026467 0.97423146 1.15174359 1.         1.14725284 1.14718193
 1.14725798 1.15170146 1.14996134 1.14723255 1.         1.15100618
 0.8649341

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [77]:
import scipy.sparse as sp
import _pickle as pkl
# L_train = pkl.load(open("train_L_S_discrete.p","rb"))
# L_train = sp.csr_matrix(L_train)

# L_gold = pkl.load(open("gold_discrete.p","rb"))
# print(np.array(L_gold).shape)
# L_gold = sp.csr_matrix(L_gold)

L_train = np.load("train_L_S_discrete.npy")
L_train = L_train[:,0,:].astype(int)
print(np.array(L_train).shape)
L_train = sp.csr_matrix(L_train)


from snorkel.learning import GenerativeModel
import time
import datetime

gen_model = GenerativeModel()

start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

gen_model.train(L_train,  cardinality=2)
# gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)


print("trained in ",str(datetime.timedelta(seconds=time.time() - start_time)))

(8272, 33)
started at: 13-6-2018, 22:20:29
trained in  0:00:28.347146


In [78]:

L_dev = np.load("dev_L_S_discrete.npy")
L_dev = L_dev[:,0,:].astype(int)
print(np.array(L_dev).shape)
L_dev = sp.csr_matrix(L_dev)

import numpy as np
marginals = gen_model.marginals(L_dev)
marginals = np.array(marginals)
print(marginals.shape)
# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(gold_labels_dev),GenLabels,average="binary"))


(888, 33)
(888,)
(888,)
(0.5381679389312977, 0.9527027027027027, 0.6878048780487805, None)


In [80]:
from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(lfs=LFs)

%time L_train = labeler.apply(split=0)

gen_model = GenerativeModel(lf_propensity=True)
gen_model.train(L_train, decay=0.95, step_size=0.1/L_train.shape[0], reg_param=0.0
)

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
MentionScorer(dev_cands, L_gold_dev).score(marginals)

Clearing existing...
Running UDF...
[=                                       ] 0%

ValueError: 
                        Unable to parse label with value (0, 0)
                        for candidate with values [True, False]

Inferred cardinality: 2
Scores (Un-adjusted)
Pos. class accuracy: 0.953
Neg. class accuracy: 0.591
Precision            0.538
Recall               0.953
F1                   0.688
----------------------------------------
TP: 282 | FP: 242 | TN: 350 | FN: 14



({ChemicalDisease(Span("b'Amiodarone'", sentence=10440, chars=[0,9], words=[0,0]), Span("b'pneumonitis'", sentence=10440, chars=[122,132], words=[18,18])),
  ChemicalDisease(Span("b'BHA'", sentence=6134, chars=[19,21], words=[5,5]), Span("b'forestomach tumors'", sentence=6134, chars=[211,228], words=[37,38])),
  ChemicalDisease(Span("b'BHA'", sentence=6138, chars=[64,66], words=[11,11]), Span("b'forestomach carcinogenesis'", sentence=6138, chars=[68,93], words=[12,13])),
  ChemicalDisease(Span("b'BHA)'", sentence=6132, chars=[84,87], words=[14,15]), Span("b'forestomach tumorigenesis'", sentence=6132, chars=[101,125], words=[19,20])),
  ChemicalDisease(Span("b'Cefotetan-induced'", sentence=9000, chars=[0,16], words=[0,0]), Span("b'hemolytic anemia'", sentence=9000, chars=[25,40], words=[2,3])),
  ChemicalDisease(Span("b'Cocaine-induced'", sentence=7230, chars=[0,14], words=[0,0]), Span("b'seizures'", sentence=7230, chars=[26,33], words=[2,2])),
  ChemicalDisease(Span("b'Cyclophosphamide

In [ ]:
L_dev = labeler.apply_existing(split=1)
_ = gen_model.error_analysis(session, L_dev, L_gold_dev)

In [74]:

L_test = np.load("test_L_S_discrete.npy")
L_test = L_test[:,0,:].astype(int)
print(np.array(L_test).shape)
L_test = sp.csr_matrix(L_test)

import numpy as np
marginals = gen_model.marginals(L_test)
marginals = np.array(marginals)
print(marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(gold_labels_test),GenLabels,average="binary"))


(4620, 33)
(4620,)
(4620,)
(0.5415730337078651, 0.9607973421926911, 0.6926946107784431, None)
